In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from seaborn import heatmap
from seaborn import pairplot
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, CategoryParameter, expand_plist, FieldSelector, Parameter, wwr
from besos.problem import EPProblem
plt.rcParams['figure.figsize'] = [8, 8]

In [3]:
filename = 'model_final_triple'
building = ef.get_building('/home/ict4bd/'+filename+'.idf')

In [4]:
[materials.Name for materials in building.idfobjects["MATERIAL"]] # get a list of the Name property of all IDF objects of class MATERIAL

['Cast Concrete (Dense)_.1',
 'Asphalt 1_.O1',
 'MW Glass Wool (rolls)_.1445',
 'Plasterboard_.O13',
 'Gypsum Plasterboard_.O25',
 'Cement/plaster/mortar - cement_.O3',
 'XPS Extruded Polystyrene  - CO2 Blowing_.2',
 'Aerated Concrete Block_.2']

In [17]:
building.idfobjects["BUILDING"][0]


Building,
    Building,                 !- Name
    0,                        !- North Axis
    Suburbs,                  !- Terrain
    0.04,                     !- Loads Convergence Tolerance Value
    0.4,                      !- Temperature Convergence Tolerance Value
    FullExterior,             !- Solar Distribution
    25,                       !- Maximum Number of Warmup Days
    6;                        !- Minimum Number of Warmup Days

In [18]:
building.idfobjects["MATERIAL"]

[
Material,
    Cast Concrete (Dense)_.1,    !- Name
    Rough,                    !- Roughness
    0.1,                      !- Thickness
    1.4,                      !- Conductivity
    2100,                     !- Density
    840,                      !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.6,                      !- Solar Absorptance
    0.6;                      !- Visible Absorptance
, 
Material,
    Asphalt 1_.O1,            !- Name
    Rough,                    !- Roughness
    0.01,                     !- Thickness
    0.7,                      !- Conductivity
    2100,                     !- Density
    1000,                     !- Specific Heat
    0.9,                      !- Thermal Absorptance
    0.85,                     !- Solar Absorptance
    0.9;                      !- Visible Absorptance
, 
Material,
    MW Glass Wool (rolls)_.1445,    !- Name
    Rough,                    !- Roughness
    0.1445,                   !- Thickness

In [6]:
insulation = FieldSelector(class_name='Material',\
                           object_name='XPS Extruded Polystyrene  - CO2 Blowing_.2',\
                           field_name='Thickness')
ach_bedroom = FieldSelector(class_name='ZoneVentilation:DesignFlowRate',object_name='Block1:Bedroom Nat Vent', field_name='Air Changes per Hour')
ach_bathroom=FieldSelector(class_name='ZoneVentilation:DesignFlowRate',object_name='Block1:Bathroom Nat Vent', field_name='Air Changes per Hour')
ach_kitchen=FieldSelector(class_name='ZoneVentilation:DesignFlowRate',object_name='Block1:Kitchen Nat Vent', field_name='Air Changes per Hour')
shading_bedroom = FieldSelector(class_name='WindowShadingControl',object_name='1001-Block1:Bedroom', field_name='Setpoint')
shading_bathroom = FieldSelector(class_name='WindowShadingControl',object_name='1002-Block1:Bathroom', field_name='Setpoint')
shading_kitchen = FieldSelector(class_name='WindowShadingControl',object_name='1003-Block1:Kitchen', field_name='Setpoint')

In [7]:
zero_to_one_exclusive = CategoryParameter(options=[0.00001, 0.10, 0.20])
ach_parameters = CategoryParameter(options=[0,5])
shading_parameters = CategoryParameter(options=[120,2000])

In [8]:
insulation_param = Parameter(selector=insulation,
                                 value_descriptor=zero_to_one_exclusive,
                                 name='Insulation Thickness')

ach_bedroom_param = Parameter(selector=ach_bedroom,
                                 value_descriptor=ach_parameters,
                                 name='Air Changes per Hour Bedroom')

ach_bathroom_param = Parameter(selector=ach_bathroom,
                                 value_descriptor=ach_parameters,
                                 name='Air Changes per Hour Bathroom')

ach_kitchen_param = Parameter(selector=ach_kitchen,
                                 value_descriptor=ach_parameters,
                                 name='Air Changes per Hour Kitchen')

shading_bedroom_param=Parameter(selector=shading_bedroom,
                                 value_descriptor=shading_parameters,
                                 name='Shading Bedroom')

shading_bathroom_param=Parameter(selector=shading_bathroom,
                                 value_descriptor=shading_parameters,
                                 name='Shading Bathroom')

shading_kitchen_param=Parameter(selector=shading_kitchen,
                                 value_descriptor=shading_parameters,
                                 name='Shading Kitchen')

In [9]:
window_to_wall = wwr(RangeParameter(0.1,0.9)) # use a special shortcut to get the window-to-wall parameter
window_to_wall

Parameter(name='Window to Wall Ratio', selector=wwrSelector(), value_descriptor=RangeParameter(min=0.1, max=0.9))

In [20]:
#previous = building.idfobjects["BUILDING"][0].obj[2]
#building.idfobjects["BUILDING"][0].obj[2] = [0.0, 90.0, 180.0, 270.0]

orientation_param = FieldSelector(class_name='Building',\
                           object_name='Building',\
                           field_name='North Axis')
orientation_parameters = CategoryParameter(options=[0.0, 90.0, 180.0, 270.0])
orientation = Parameter(selector = orientation_param, value_descriptor = orientation_parameters, name = 'Orientation')

orientation

Parameter(name='Orientation', selector=FieldSelector(field_name='North Axis', class_name='Building', object_name='Building'), value_descriptor=CategoryParameter(options=[0.0, 90.0, 180.0, 270.0]))

In [21]:
# Add all the parameters to a single parameters object.
#parameters = [insulation_param] + [window_to_wall]
parameters = [window_to_wall,insulation_param,orientation,ach_bedroom_param,ach_bathroom_param,ach_kitchen_param,shading_bedroom_param,shading_bathroom_param,shading_kitchen_param] 

# Let us try to optimize the Cooling and the Heating of facility at the same time
objectives = ['Electricity:Facility','DistrictCooling:Facility'] 
#'InteriorLights:Electricity' has correlation circa 1 with ELectricity:Facility, we remove it from the obj
# These are hourly values, default is sum all the hourly values

# Construct the problem 
problem = EPProblem(parameters, objectives) # Make a problem instance from the parameters and objectives

In [22]:
samples = sampling.dist_sampler(sampling.lhs, problem, num_samples=12)

#samples = pd.read_csv(filename+'.csv').drop(['Electricity:Facility','DistrictCooling:Facility'], axis=1)
samples

,Window to Wall Ratio,Insulation Thickness,Orientation,Air Changes per Hour Bedroom,Air Changes per Hour Bathroom,Air Changes per Hour Kitchen,Shading Bedroom,Shading Bathroom,Shading Kitchen
0,0.201992,0.20000,270.0,0,5,5,2000,120,120
1,0.549148,0.00001,0.0,5,5,5,120,2000,120
2,0.870987,0.00001,90.0,0,5,0,120,2000,2000
3,0.800321,0.10000,90.0,0,5,0,2000,2000,120
4,0.443826,0.10000,0.0,5,5,0,2000,120,120
5,0.353736,0.10000,90.0,0,0,5,120,2000,2000
6,0.600416,0.10000,180.0,5,5,5,120,2000,2000
7,0.762949,0.20000,180.0,5,0,0,120,120,2000
8,0.255613,0.20000,270.0,5,0,0,2000,120,2000
9,0.121195,0.00001,0.0,5,0,0,120,120,2000


In [ ]:
evaluator = EvaluatorEP(problem, building, out_dir='outputdir', err_dir='outputdir',epw='/home/ict4bd/larnaca.epw')
outputs = evaluator.df_apply(samples, keep_input=True)
outputs.describe()

In [ ]:
_ = outputs.plot(x=objectives[0], y=objectives[1], style = 'o')
plt.xlabel(objectives[0])
plt.ylabel(objectives[1])

In [ ]:
outputs = outputs.sort_values(by=objectives[1])
ax=outputs.plot.bar(subplots=True,legend=None, figsize=(8,15))
outputs.to_csv(filename + '.csv', index = False)

In [ ]:
outputs